In [1]:
%run ~/relmapping/annot/notebooks/annot__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/beegfs/scratch_copy/ahringer/jj374/lab/relmapping


In [2]:
df_regl = df_regl_()

In [3]:
def name_(annot_fwd, annot_rev, locus_id_fwd, locus_id_rev, enhancer_locus_id):
    if (annot_fwd in config['annot_with_gene_id']) and not(annot_rev in config['annot_with_gene_id']):
        return locus_id_fwd
    elif not(annot_fwd in config['annot_with_gene_id']) and (annot_rev in config['annot_with_gene_id']):
        return locus_id_rev
    elif (annot_fwd in config['annot_with_gene_id']) and (annot_rev in config['annot_with_gene_id']):
        return '%s / %s' % (locus_id_rev, locus_id_fwd)
    elif enhancer_locus_id != '.': # show "likeliest target gene for an enhancer" in brackets for all remaining (=not coding_promoter or non_coding_promoter)
        return '(%s)' % (enhancer_locus_id,)
    else:
        return ''

df_regl['label'] = list(map(name_,
    df_regl['annot_fwd'], df_regl['annot_rev'],                     
    df_regl['promoter_locus_id_fwd'], df_regl['promoter_locus_id_rev'],
    df_regl['associated_locus_id']
))

In [4]:
# Table S2 -- unstranded summary annotation


d_annot_strand_legend = collections.OrderedDict([
    ('coding_promoter', yp.RED),
    ('pseudogene_promoter', yp.ORANGE),
    ('non-coding_RNA', yp.BLACK),
    ('unknown_promoter', yp.YELLOW),
    ('transcription_initiation', yp.GREEN),
    ('no_transcription', yp.BLUE),
])

d_annot_legend = collections.OrderedDict([
    ('coding_promoter', yp.RED),
    ('pseudogene_promoter', yp.ORANGE),
    ('non-coding_RNA', yp.BLACK),
    ('unknown_promoter', yp.YELLOW),
    ('putative_enhancer', yp.GREEN),
    ('other_element', yp.BLUE),
])

write_gffbed('annot_eLife_revised/reg_elements_ce10_split_fwd.bed',
    chrom = df_regl['chrom'],
    start = df_regl['pos'],
    end = df_regl['end'],
    name = '',
    strand = '.',#df_regl['strand'],
    itemRgb = list(map(lambda annot: d_annot_strand_legend[annot], df_regl['annot_fwd'])),
    attr = df_regl[['annot', 'annot_fwd', 'annot_detailed_fwd']],
    trackline = None,
)

write_gffbed('annot_eLife_revised/reg_elements_ce10_split_rev.bed',
    chrom = df_regl['chrom'],
    start = df_regl['start'],
    end = df_regl['end'],
    name = df_regl['label'],
    strand = '.',#df_regl['strand'],
    itemRgb = list(map(lambda annot: d_annot_strand_legend[annot], df_regl['annot_rev'])),
    attr = df_regl[['annot', 'annot_rev', 'annot_detailed_rev']],
    trackline = None,
)

!cat annot_eLife_revised/reg_elements_ce10_split_fwd.bed annot_eLife_revised/reg_elements_ce10_split_rev.bed \
| sort -k1,1 -k2,2n - > annot_eLife_revised/reg_elements_eLife_revised_ce10_twocolour.bed

!sed -i '1i#track gffTags=on' annot_eLife_revised/reg_elements_eLife_revised_ce10_twocolour.bed

!rm annot_eLife_revised/reg_elements_ce10_split_fwd.bed annot_eLife_revised/reg_elements_ce10_split_rev.bed